# Using LLM Evaluation with Langchain

*This cookbook was created from a Jypyter notebook which can be found [here](TBD).*

For this guide, we will be performing a text to sql query generation task using LangChain. We will be using the Chinook database which contains the SQLite database of a music store with both employee, customer and invoice data.

We will highlight three different parts of the workflow:

1. Creating a synthetic dataset of questions
2. Creating a LangChain chain to generate SQL queries
3. Automating the evaluation of the SQL queries on the synthetic dataset

## Preparing our environment

First, we will install the necessary libraries, download the Chinook database and set up our different API keys.

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Download the relevant data
import os
from langchain_community.utilities import SQLDatabase

import requests
import os

url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
filename = "Chinook_Sqlite.sqlite"

if not os.path.exists(filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"Chinook database downloaded")
    
db = SQLDatabase.from_uri(f"sqlite:///{filename}")

Chinook database downloaded


In [2]:
import os
import getpass

os.environ["COMET_URL_OVERRIDE"] = "http://localhost:5173/api"

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

## Creating a synthetic dataset

In order to create our synthetic dataset, we will be using the OpenAI API to generate 20 different questions that a user might ask based on the Chinook database.

In order to ensure that the OpenAI API calls are being tracked, we will be using the `track_openai` function from the `opik` library.

In [3]:
from opik.integrations.openai import track_openai
from openai import OpenAI
import json

os.environ["COMET_PROJECT_NAME"] = "openai-integration"
client = OpenAI()

openai_client = track_openai(client)

prompt = """
Create 20 different example questions a user might ask based on the Chinook Database.

These questions should be complex and require the model to think. They should include complex joins and window functions to answer.

Return the response as a json object with a "result" key and an array of strings with the question.
"""

completion = openai_client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message.content)

{
  "result": [
    "Which customer has made the most purchases in terms of total dollars spent?",
    "What is the total number of tracks sold in each genre?",
    "How many unique albums have been purchased by customers from different countries?",
    "Which employee sold the most expensive track?",
    "What is the average length of tracks purchased by customers from each country?",
    "Which customer has spent the most money on tracks in the rock genre?",
    "What is the total revenue generated by each employee?",
    "How many unique artists are featured in each playlist?",
    "Which customer has the highest average rating on their purchased tracks?",
    "What is the total value of invoices generated by each sales support agent?",
    "How many tracks have been sold to customers in each country?",
    "Which artist has the most tracks featured in the top 100 selling tracks?",
    "What is the total value of invoices generated in each year?",
    "How many unique tracks have be

Now that we have our synthetic dataset, we can create a dataset in Comet and insert the questions into it.

In [4]:
# Create the synthetic dataset
from opik import Opik
from opik import DatasetItem

synthetic_questions = json.loads(completion.choices[0].message.content)["result"]

client = Opik()
try:
    dataset = client.create_dataset(name="synthetic_questions")
    dataset.insert([
        DatasetItem(input={"question": question}) for question in synthetic_questions
    ])
except Exception as e:
    pass

## Creating a LangChain chain

We will be using the `create_sql_query_chain` function from the `langchain` library to create a SQL query to answer the question.

We will be using the `CometTracer` class from the `opik` library to ensure that the LangChan trace are being tracked in Comet.

In [7]:
# Use langchain to create a SQL query to answer the question
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
from opik.integrations.langchain import OpikTracer

os.environ["COMET_PROJECT_NAME"] = "sql_question_answering"
opik_tracer = OpikTracer(tags=["simple_chain"])

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there ?"}, {"callbacks": [opik_tracer]})
response

print(response)

SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM "Employee"


## Automatting the evaluation

In order to ensure our LLM application is working correctly, we will test it on our synthetic dataset.

For this we will be using the `evaluate` function from the `opik` library. We will evaluate the application using a custom metric that checks if the SQL query is valid.

In [8]:
from opik import Opik, track
from opik.evaluation import evaluate
from opik.evaluation.metrics import Contains


contains_hello = Contains(name="ContainsHello")

client = Opik()
dataset = client.get_dataset("synthetic_questions")

@track()
def llm_chain(input):
    opik_tracer = OpikTracer(tags=["simple_chain"])

    db = SQLDatabase.from_uri("sqlite:///Chinook_Sqlite.sqlite")
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    chain = create_sql_query_chain(llm, db)
    response = chain.invoke({"question": input}, {"callbacks": [opik_tracer]})
    
    return response

def evaluation_task(item):
    response = llm_chain(item.input["question"])

    return {
        "reference": "hello",
        "output": response
    }

res = evaluate(
    experiment_name="sql_question_answering_v2",
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=[contains_hello]
)

Scoring outputs: 100%|██████████| 20/20 [00:00<00:00, 82321.96it/s]


╭─ synthetic_questions (20 samples) ─╮
│                                    │
│ Total time:        00:00:03        │
│ Number of samples: 20              │
│                                    │
│ ContainsHello: 0.0000 (avg)        │
│                                    │
╰────────────────────────────────────╯

Uploading results to Opik ...